In [ ]:
# example of loading the mnist dataset
from numpy import mean
from numpy import std
from sklearn import datasets
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.optimizers import SGD
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
#from tensorflow.keras.datasets import cifar10,cifar100
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import keras
import numpy as np
from .model import *
def conc(*inp1):
  return layers.Concatenate()(inp1)

#!pip install tensorflow_addons
import tensorflow_addons as tfa

def mpool(psize,strides=2):
  return MaxPooling2D(pool_size=psize,strides=strides,padding="VALID")

def apool(psize,strides=None):
  if(strides is None):
    return AveragePooling2D(pool_size=psize,padding='SAME')
  else:
    return AveragePooling2D(pool_size=psize,strides=strides,padding="SAME")

def ln():
  return layers.LayerNormalization()

def bn():
  return layers.BatchNormalization()

from keras.callbacks import ModelCheckpoint
import tensorflow.keras.layers as layers

learning_rate = 0.0001
lr_drop=7
def lr_scheduler(epoch):
        return learning_rate * (0.5 ** (epoch // lr_drop))

reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

def train(model,path,epochs=100):

  checkpoint1 = ModelCheckpoint(filepath='./'+path,save_format=tf,monitor='val_loss',
                            save_weights_only=True,
                             verbose=1,
                             save_best_only=True,
                             mode='min')
  model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate,momentum=0.9,clipvalue=0.01,global_clipnorm=0.01),
    loss='categorical_crossentropy',
    metrics='categorical_accuracy')
  model.fit(trainXA, trainY, epochs=epochs, batch_size=200, validation_data=(testXA, testY), callbacks=[checkpoint1, reduce_lr], verbose=1)

def bnconv(inp,units,kernel_size):
  return BatchNormalization()(Conv2D(units,kernel_size,activation='relu',kernel_regularizer=tf.keras.regularizers.L2(0.0005))(inp))

def dense(units,act='relu'):
  return layers.Dense(units,activation=act)

def conv(inp,units,kernel_size):
  return Conv2D(kernel_size,units,activation='relu',kernel_regularizer=tf.keras.regularizers.L2(0.0005))(inp)

import numpy as np

def preprocess_imagecont(image, label):
  image = tf.convert_to_tensor(image)
  #image = tf.image.grayscale_to_rgb(image,name=None)
  image = tf.image.resize(image, [28,28])
  image = tf.cast(image, tf.float32)
  image1 = tfa.image.gaussian_filter2d(image, (2,2),3)
  positionsx1 = tf.range(start=0, limit=28, delta=1,dtype=tf.float32)
  positionsy1 = tf.range(start=0, limit=28, delta=1,dtype=tf.float32)
  positionsx1 = tf.expand_dims(tf.tile(tf.expand_dims(positionsx1,0),[28,1]),-1)
  positionsy1 = tf.expand_dims(tf.tile(tf.transpose(tf.expand_dims(positionsy1,0)),[1,28]),-1)
  positions11 = tf.concat([positionsx1,positionsy1],-1); positions11 = tf.tile(positions11[tf.newaxis,:,:,:],[image.shape[0],1,1,1])
  u = tf.image.sobel_edges(image1)
  angle = tf.where(u[:,:,:,0,0]!=0,tf.atan2(u[:,:,:,0,1],u[:,:,:,0,0]),0)
  angle = angle[:,:,:,tf.newaxis]
  #print(angle[0])
  u = tf.squeeze(u,3)
  image = tf.concat([image,angle/3.14,positions11],-1) # ,angle/3.14,positions11
  return image, label

def load_dataset():
	# load dataset
  (trainX, trainY), (testX, testY) = mnist.load_data()
  trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
  testX = testX.reshape((testX.shape[0], 28, 28, 1))
  return trainX, trainY, testX, testY

def prep_pixels(train, test):
	# convert from integers to floats
  train_norm = train.astype('float32')
  test_norm = test.astype('float32')
	# normalize to range 0-1
  train_norm = train_norm / 255.0
  test_norm = test_norm / 255.0
  normalization = layers.Normalization()
  normalization.adapt(train_norm)
  #train_norm=normalization(train_norm)
  #test_norm=normalization(test_norm)
  return train_norm, test_norm

trainX, trainY, testX, testY = load_dataset()
	# prepare pixel data
trainX, testX = prep_pixels(trainX, testX)
trainXA,trainY = preprocess_imagecont(trainX,trainY)
testXA,testY = preprocess_imagecont(testX,testY)
#train_images_res = tf.image.resize(trainXA,[48,48],method='bilinear')
#test_images_res = tf.image.resize(testXA,[48,48],method='bilinear')

trainY = tf.keras.utils.to_categorical(trainY,num_classes=10)
testY = tf.keras.utils.to_categorical(testY,num_classes=10)

C:\Users\arpit.rai\.conda\envs\tf-gpu\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\arpit.rai\.conda\envs\tf-gpu\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
I

In [ ]:
learning_rate = 0.0001
def train(model,path,epochs=100,learning_rate=0.00014):
  checkpoint1 = ModelCheckpoint(filepath='./'+path,save_format=tf,monitor='val_loss',
                            save_weights_only=True,
                             verbose=1,
                             save_best_only=True,
                             mode='min')
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), #,momentum=0.9#,clipvalue=0.01,global_clipnorm=0.01
    loss='categorical_crossentropy',
    metrics='categorical_accuracy')
  model.fit(trainXA, trainY, epochs=epochs, batch_size=200, validation_data=(testXA, testY), callbacks=[checkpoint1, reduce_lr], verbose=1)

In [ ]:
import keras.backend as K

In [ ]:
import math

In [ ]:
learning_rate=0.0001

In [ ]:
def train(model,path,epochs=200,learning_rate=0.1):
  checkpoint1 = ModelCheckpoint(filepath='./'+path,save_format=tf,monitor='val_loss',
                            save_weights_only=True,
                             verbose=1,
                             save_best_only=True,
                             mode='min')
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),#.SGD(learning_rate,momentum=0.9),
    loss='categorical_crossentropy',
    metrics='categorical_accuracy')
  model.fit(trainXA, trainY, epochs=epochs, batch_size=64, validation_data=(testXA, testY), callbacks=[checkpoint1, reduce_lr], verbose=1)

In [ ]:
from keras.regularizers import L1,L2

In [ ]:
from operator import itemgetter

In [ ]:
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
import keras.layers as layers

val_loss = tf.keras.metrics.Mean()

class RobustPrototypicalTraining(keras.Model):
    def __init__(self,encoder,cls_model,decoder,epoch=0,size=512,nlayers=2,shape=8):
        super(RobustPrototypicalTraining, self).__init__()
        self.supervised=True; self.num_outputs = 100
        shape=28
        cls_model = cls_model
        self.classifier = cls_model;
        #if rec:
        self.decoder = decoder
        self.encoder = encoder;
        self.cls_loss = tf.keras.losses.CategoricalCrossentropy(); self.accuracy = tf.keras.metrics.CategoricalAccuracy()

    def compile(self, optimizer):
        super().compile(optimizer)
        self.optimizer = optimizer

    def set_supervised(self):
        self.supervised=True

    def train_step(self, data):
        X, label = data
        with tf.GradientTape(persistent=True) as tape:
          pred = self.classifier(X); out = self.encoder(X);
          cls_loss = tf.reduce_mean(self.cls_loss(label,pred))
          accuracy = self.accuracy(label,pred)
          reconstruction_loss = 0.07*tf.reduce_mean(tf.math.abs(X-self.decoder(X)))
        grads = tape.gradient(cls_loss, self.classifier.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.classifier.trainable_variables))
        grads = tape.gradient(reconstruction_loss, self.decoder.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.decoder.trainable_variables))
        return {"loss":reconstruction_loss,'cls_loss':cls_loss,"acc": accuracy}

    def test_step(self, data):
        X, label = data
        pred = self.encoder(X); cls = self.classifier(X)
        cls_loss = tf.reduce_mean(self.cls_loss(label,cls)) #+ 0.006*tf.reduce_sum(tf.square(self.classifier.get_layer(name='margin_dense').kernel))#**2
        reconstruction_loss = tf.reduce_mean(tf.math.abs(X-self.mid(X)))

        accuracy = self.accuracy(label,cls)
        return {"lossi":reconstruction_loss,'cls_loss':cls_loss,"acc": accuracy} #total_loss

In [ ]:
checkpoint3 = ModelCheckpoint(filepath='mnist_rbf_robustrec_07_07(2)',save_format=tf,monitor='val_cls_loss', #mnist_simple_rbf_26.1
                            save_weights_only=True,
                             verbose=1,
                             save_best_only=True,
                             mode='min')

# Robustness Experiments with vanilla softmax cnn Model

In [ ]:
cms1 = VanillaSoftmaxModel()
cms1.compile(loss='categorical_crossentropy',metrics='categorical_accuracy')

In [ ]:
cms1.evaluate(testXA,testY)

313/313 [==============================] - 4s 10ms/step - loss: 0.4014 - categorical_accuracy: 0.9945


[0.4014187455177307, 0.9945000410079956]

In [ ]:
train(cms1,epochs=40,path='Softmax_4C_MNIST2')

Epoch 1/40
937/938 [============================>.] - ETA: 0s - loss: 0.6129 - categorical_accuracy: 0.9669WARNING:tensorflow:Model was constructed with shape (None, 28, 28, 1) for input KerasTensor(type_spec=TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='input_9'), name='input_9', description="created by layer 'input_9'"), but it was called on an input with incompatible shape (None, 28, 28, 4).

Epoch 1: val_loss improved from inf to 0.51337, saving model to .\Softmax_4C_MNIST2
938/938 [==============================] - 30s 30ms/step - loss: 0.6128 - categorical_accuracy: 0.9669 - val_loss: 0.5134 - val_categorical_accuracy: 0.9845 - lr: 1.0000e-04
Epoch 2/40
937/938 [============================>.] - ETA: 0s - loss: 0.4601 - categorical_accuracy: 0.9900
Epoch 2: val_loss improved from 0.51337 to 0.42767, saving model to .\Softmax_4C_MNIST2
938/938 [==============================] - 27s 29ms/step - loss: 0.4602 - categorical_accuracy: 0.9900 - val_loss: 0.4277 - val_categ

In [ ]:
cms1.load_weights('Softmax_4C_MNIST2')

# Accuracy against noise Of Softmax CNN Model

In [ ]:
test_different_noise_levels(cms1)

Original_accuracy:
313/313 [==============================] - 3s 10ms/step - loss: 0.4014 - categorical_accuracy: 0.9945
Accuracy at noise level 0.4:
313/313 [==============================] - 3s 9ms/step - loss: 2.1990 - categorical_accuracy: 0.5223
Accuracy at noise level 0.5:
313/313 [==============================] - 3s 9ms/step - loss: 3.6871 - categorical_accuracy: 0.2747
Accuracy at noise level 0.8:
313/313 [==============================] - 3s 10ms/step - loss: 6.6495 - categorical_accuracy: 0.0974
Accuracy at noise level 1.0:
313/313 [==============================] - 3s 9ms/step - loss: 6.6550 - categorical_accuracy: 0.0975


##Accuracy against FGSM attack of Softmax CNN

In [ ]:
test_adversarial_robustness_fgsm(cms1)

Original_accuracy:
313/313 [==============================] - 7s 23ms/step - loss: 0.4014 - categorical_accuracy: 0.9945
99.45000410079956%
Accuracy at noise level 0.08:
(5000, 28, 28, 4)
157/157 [==============================] - 2s 14ms/step - loss: 1.2199 - categorical_accuracy: 0.8034
(5000, 28, 28, 4)
157/157 [==============================] - 2s 13ms/step - loss: 0.8249 - categorical_accuracy: 0.8876
84.55000519752502%
Accuracy at noise level 0.1:
(5000, 28, 28, 4)
157/157 [==============================] - 2s 13ms/step - loss: 1.7378 - categorical_accuracy: 0.6880
(5000, 28, 28, 4)
157/157 [==============================] - 2s 15ms/step - loss: 1.2157 - categorical_accuracy: 0.7840
73.60000312328339%
Accuracy at noise level 0.15:
(5000, 28, 28, 4)
157/157 [==============================] - 2s 14ms/step - loss: 3.6681 - categorical_accuracy: 0.2874
(5000, 28, 28, 4)
157/157 [==============================] - 2s 14ms/step - loss: 2.9839 - categorical_accuracy: 0.3506
31.9000005722

# Robustness Experiments with our **Robust Large Margin Prototypical Encoder**

Model Training...

In [ ]:
cls,mid,decoder = RobustMarginPrototypeModel()

In [ ]:
learning_rate=0.0001
svr6 = SparseVarianceReduction(cls_model=cls,encoder=mid,decoder=decoder)
svr6.compile(optimizer=tf.keras.optimizers.Adam(0.0001,clipvalue=1.0,clipnorm=1.0))
svr6.fit(trainX,trainY,validation_data=(testX,testY),epochs=40,callbacks=[checkpoint3,reduce_lr],batch_size=64)

Epoch 1/40
epch0
epch0
938/938 [==============================] - ETA: 0s - loss: 0.0095 - cls_loss: 0.1655 - acc: 0.8863
Epoch 1: val_cls_loss improved from inf to 0.00287, saving model to mnist_rbf_robustrec_07_07(2)
938/938 [==============================] - 71s 67ms/step - loss: 0.0095 - cls_loss: 0.1654 - acc: 0.8863 - val_lossi: 0.1624 - val_cls_loss: 0.0029 - val_acc: 0.9853 - lr: 1.0000e-04
Epoch 2/40
938/938 [==============================] - ETA: 0s - loss: 0.0091 - cls_loss: 0.0400 - acc: 0.9873
Epoch 2: val_cls_loss improved from 0.00287 to 0.00083, saving model to mnist_rbf_robustrec_07_07(2)
938/938 [==============================] - 58s 62ms/step - loss: 0.0091 - cls_loss: 0.0400 - acc: 0.9873 - val_lossi: 0.1624 - val_cls_loss: 8.3311e-04 - val_acc: 0.9881 - lr: 1.0000e-04
Epoch 3/40
938/938 [==============================] - ETA: 0s - loss: 0.0091 - cls_loss: 0.0252 - acc: 0.9925
Epoch 3: val_cls_loss improved from 0.00083 to 0.00028, saving model to mnist_rbf_robustre

In [ ]:
svr6.load_weights('mnist_rbf_robustrec_07_07(2)')

### Evaluating robustness to noise of different levels

In [ ]:
test_different_noise_levels(svr6)

Original_accuracy:
313/313 [==============================] - 7s 17ms/step - lossi: 6.8303 - cls_loss: 0.0289 - acc: 0.9934
Accuracy at noise level 0.4:
313/313 [==============================] - 5s 15ms/step - lossi: 7.0135 - cls_loss: 0.0335 - acc: 0.9927
Accuracy at noise level 0.5:
313/313 [==============================] - 5s 15ms/step - lossi: 7.0599 - cls_loss: 0.0415 - acc: 0.9895
Accuracy at noise level 0.8:
313/313 [==============================] - 4s 14ms/step - lossi: 7.2007 - cls_loss: 0.1839 - acc: 0.9427
Accuracy at noise level 1.0:
313/313 [==============================] - 5s 15ms/step - lossi: 7.2008 - cls_loss: 0.1865 - acc: 0.9430


### FGSM attacks of different noise levels (epsilons) on our Robust Large Margin Prototypical Encoder (RMPE)

In [ ]:
test_adversarial_robustness_fgsm(svr6)

Original_accuracy:
313/313 [==============================] - 7s 24ms/step - lossi: 6.8303 - cls_loss: 0.0289 - acc: 0.9934
99.52000379562378%
Accuracy at noise level 0.08:
(5000, 28, 28, 4)
157/157 [==============================] - 3s 21ms/step - lossi: 6.8421 - cls_loss: 0.8609 - acc: 0.8859
(5000, 28, 28, 4)
157/157 [==============================] - 3s 19ms/step - lossi: 6.8498 - cls_loss: 0.3757 - acc: 0.9490
91.31000339984894%
Accuracy at noise level 0.1:
(5000, 28, 28, 4)
157/157 [==============================] - 3s 20ms/step - lossi: 6.8461 - cls_loss: 1.3622 - acc: 0.8357
(5000, 28, 28, 4)
157/157 [==============================] - 3s 18ms/step - lossi: 6.8537 - cls_loss: 0.6389 - acc: 0.9105
86.59000396728516%
Accuracy at noise level 0.15:
(5000, 28, 28, 4)
157/157 [==============================] - 3s 20ms/step - lossi: 6.8562 - cls_loss: 3.2546 - acc: 0.5750
(5000, 28, 28, 4)
157/157 [==============================] - 3s 18ms/step - lossi: 6.8635 - cls_loss: 1.9737 - acc:

In [ ]:
svr6.load_weights('mnist_rbf_robustrec_07_07(2)')

#Function to generate Adversarial Example And Noise

In [ ]:
def create_adversarial_pattern(model,input_image, input_label,epsilon=0.02):
  with tf.GradientTape() as tape:
    tape.watch(input_image)
    prediction = model(input_image)
    loss = loss_object(input_label, prediction)

  # Get the gradients of the loss w.r.t to the input image.
  gradient = tape.gradient(loss, input_image)
  # Get the sign of the gradients to create the perturbation
  signed_grad = tf.sign(gradient)
  print(signed_grad.shape)
  input_image_perturbed = signed_grad*epsilon + input_image
  r = model.evaluate(input_image_perturbed,input_label)
  #print(r)
  return r
loss_object = keras.losses.CategoricalCrossentropy()

In [ ]:
def test_noise_pattern(model,input_image, input_label,epsilon=0.8,cutout=False,cutsize=1.0):

  #input_image_perturbed = signed_grad*epsilon + input_image
  noise = epsilon*tf.random.uniform(shape=input_image.shape)
  if cutout:
      size = int(28*cutsize)
      masked_noise = tfa.image.cutout(images=noise,mask_size=size,offset=(10,10))
      print(masked_noise.shape)
      input_image_perturbed = masked_noise + input_image
  else:
      input_image_perturbed = noise + input_image
  r = model.evaluate(input_image_perturbed,input_label)
  #print(r)
  return r
loss_object = tf.keras.losses.CategoricalCrossentropy()

In [ ]:
def test_different_noise_levels(model):
    print('Original_accuracy:')
    model.evaluate(testXA,testY)
    print('Accuracy at noise level 0.4:')
    test_noise_pattern(model,input_image=testXA,input_label=testY,epsilon=0.4)
    print('Accuracy at noise level 0.5:')
    test_noise_pattern(model,input_image=testXA,input_label=testY,epsilon=0.5)
    print('Accuracy at noise level 0.8:')
    test_noise_pattern(model,input_image=testXA,input_label=testY,epsilon=0.8)
    print('Accuracy at noise level 1.0:')
    test_noise_pattern(model,input_image=testXA,input_label=testY,epsilon=0.8)

In [ ]:
def test_adversarial_robustness_fgsm(model):
    print('Original_accuracy:')
    print(str(model.evaluate(testXA,testY)[1]*100)+'%')
    print('Accuracy at noise level 0.08:')
    x1 = create_adversarial_pattern(model,input_image=testXA[:5000],input_label=testY[:5000],epsilon=0.08)[1]
    x2 = create_adversarial_pattern(model,input_image=testXA[5000:],input_label=testY[5000:],epsilon=0.08)[1]
    print(str(100*(x1+x2)/2)+'%')
    print('Accuracy at noise level 0.1:')
    x1 = create_adversarial_pattern(model,input_image=testXA[:5000],input_label=testY[:5000],epsilon=0.1)[1]
    x2 = create_adversarial_pattern(model,input_image=testXA[5000:],input_label=testY[5000:],epsilon=0.1)[1]
    print(str(100*(x1+x2)/2)+'%')
    print('Accuracy at noise level 0.15:')
    x1 = create_adversarial_pattern(model,input_image=testXA[:5000],input_label=testY[:5000],epsilon=0.15)[1]
    x2 = create_adversarial_pattern(model,input_image=testXA[5000:],input_label=testY[5000:],epsilon=0.15)[1]
    print(str(100*(x1+x2)/2)+'%')
    print('Accuracy at noise level 0.25:')
    x1 = create_adversarial_pattern(model,input_image=testXA[:5000],input_label=testY[:5000],epsilon=0.25)[1]
    x2 = create_adversarial_pattern(model,input_image=testXA[5000:],input_label=testY[5000:],epsilon=0.25)[1]
    print(str(100*(x1+x2)/2)+'%')

In [ ]:
def create_adversarial_pattern(model,input_image, input_label,epsilon=0.02,cutout=False,cutsize=1.0):

  #input_image_perturbed = signed_grad*epsilon + input_image
  noise = 0.8*tf.random.uniform(shape=input_image.shape)
  if cutout:
      size = int(28*cutsize)
      masked_noise = tfa.image.random_cutout(images=noise,mask_size=size)
      print(masked_noise.shape)
      input_image_perturbed = masked_noise + input_image
  else:
      input_image_perturbed = noise + input_image
  r = model.evaluate(input_image_perturbed,input_label)[0]
  #print(r)
  return r
loss_object = tf.keras.losses.CategoricalCrossentropy()